In [1]:
import pandas as pd

ia_P = pd.read_csv("/data/home/shared/onestop/new/ia_P.tsv", sep="\t", engine="pyarrow")


In [2]:
trial_key_cols = [
    "RECORDING_SESSION_LABEL",
    "batch",
    "article_id",
    "paragraph_id",
    "reread",
]

unknown_features = [
    # "Session_Name_", #! Discard: Equivilant to `RECORDING_SESSION_LABEL` column
    # "DATA_FILE",  #! Discard: Equivilant to `RECORDING_SESSION_LABEL` column, but with `.edf` suffix
    # "Trial_Recycled_", #! Discard: Always false
    # "LETTER_HIGHT",  #! Discard: Always 38 - appears in the paper
    # "LETTER_WIDTH",  #! Discard: Always 19 - appears in the paper
    # "DUMMY", #! Discard: always NaN
    # "TRIAL_INDEX", #! Discard:
    "COMPREHENSION_PERCENT",  # ?
    "COMPREHENSION_SCORE",  # ?
    "CONFIRM_FINAL_ANSWER_RT",  # ?
    "EXPERIMENT_DURATION",  # ?
    "SELECT_FINAL_ANSWER_RT",  # ?
    "TRIGGER_PADDING_X",  # TODO Always 10 - Add to paper?
    "TRIGGER_PADDING_Y",  # TODO Always 5 - Add to paper?
    "TOP_LEFT",  # Problematic: Unique values: `(368, 186)` 88%, `(368,186)` 12%, `UNDEFINED` <1%
    "NUM_RECALIBRATIONS",
    "PARAGRAPH_RT",
    "PREVIEW_QUESTION_RT",
    "QUESTION_RT",
    "RECALIBRATE",
    "ALL_ANSWERS",
    "ANSWER",
    "ANSWER_LOCATIONS",
    "ANSWER_PRESS_NUMBER",
    "ANSWER_RT",
    "INTEREST_AREA_FIXATION_SEQUENCE",
    "SELECT_TO_CONFIRM_RT",
]

In [3]:
import numpy as np

unknwon_features_df = ia_P[unknown_features + trial_key_cols]

In [4]:
# Goal: show that all unknown features are constant within a trial (trial level features)
value_count_df = unknwon_features_df.groupby(trial_key_cols).count()

print(np.sum([len(np.unique(x)) != 1 for x in value_count_df.values]))


0


After we have shown that all these features are constant within each trial, we can groupby and keep the first value of each feature.

In [5]:
unknwon_features_df = unknwon_features_df.groupby(trial_key_cols).first().reset_index()

In [6]:
unknwon_features_df


,RECORDING_SESSION_LABEL,batch,article_id,paragraph_id,reread,COMPREHENSION_PERCENT,COMPREHENSION_SCORE,CONFIRM_FINAL_ANSWER_RT,EXPERIMENT_DURATION,SELECT_FINAL_ANSWER_RT,...,PREVIEW_QUESTION_RT,QUESTION_RT,RECALIBRATE,ALL_ANSWERS,ANSWER,ANSWER_LOCATIONS,ANSWER_PRESS_NUMBER,ANSWER_RT,INTEREST_AREA_FIXATION_SEQUENCE,SELECT_TO_CONFIRM_RT
0,L41_408,3,0,1,0,100,1,15098,8,13569,...,0,5162,False,"[('3', 9422), ('0', 12230), ('2', 13087), ('1'...",1,"[(930, 414), (180, 756), (1680, 756), (930, 10...",4,13569,"[1, 2, 3, 5, 7, 9, 2, 1, 4, 6, 9, 11, 10, 13, ...",1528
1,L41_408,3,0,2,0,50,1,6388,9,4342,...,0,3053,False,"[('3', 9422), ('0', 12230), ('2', 13087), ('1'...",2,"[(930, 414), (180, 756), (1680, 756), (930, 10...",3,4342,"[1, 3, 6, 4, 3, 9, 11, 12, 14, 16, 8, 5, 3, 2,...",2046
2,L41_408,3,1,1,0,76,23,2615,24,2335,...,0,1957,False,"[('3', 9422), ('0', 12230), ('2', 13087), ('1'...",2,"[(930, 414), (180, 756), (1680, 756), (930, 10...",2,2335,"[1, 3, 5, 8, 12, 13, 16, 18, 21, 22, 23, 25, 2...",279
3,L41_408,3,1,2,0,77,24,3602,24,3318,...,0,1199,False,"[('3', 9422), ('0', 12230), ('2', 13087), ('1'...",2,"[(930, 414), (180, 756), (1680, 756), (930, 10...",2,3318,"[1, 4, 3, 6, 7, 8, 9, 8, 6, 11, 14, 13, 15, 17...",283
4,L41_408,3,1,3,0,75,24,10750,25,10270,...,0,2187,False,"[('3', 9422), ('0', 12230), ('2', 13087), ('1'...",1,"[(930, 414), (180, 756), (1680, 756), (930, 10...",2,10270,"[1, 4, 2, 7, 10, 12, 16, 17, 19, 20, 21, 25, 2...",479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24041,l9_536,3,10,2,0,92,38,4994,48,4683,...,1024,1160,False,"[('0', 16192), ('3', 11069), ('3', 3718), ('3'...",3,"[(930,414), (180,756), (1680,756), (930,1098)]",1,4683,"[1, 3, 4, 5, 8, 9, 0, 14, 18, 16, 19, 21, 24, ...",310
24042,l9_536,3,10,3,0,92,39,2962,48,2527,...,2301,1256,False,"[('0', 16192), ('3', 11069), ('3', 3718), ('3'...",2,"[(930,414), (180,756), (1680,756), (930,1098)]",1,2527,"[1, 2, 3, 4, 5, 7, 10, 12, 15, 18, 21, 23, 25,...",435
24043,l9_536,3,10,4,0,93,40,2574,49,2179,...,1811,1147,False,"[('0', 16192), ('3', 11069), ('3', 3718), ('3'...",3,"[(930,414), (180,756), (1680,756), (930,1098)]",1,2179,"[1, 2, 3, 4, 5, 6, 7, 9, 11, 13, 17, 16, 19, 2...",395
24044,l9_536,3,10,5,0,93,41,4714,50,3506,...,1892,1023,False,"[('0', 16192), ('3', 11069), ('3', 3718), ('3'...",3,"[(930,414), (180,756), (1680,756), (930,1098)]",1,3506,"[1, 2, 4, 6, 10, 13, 15, 18, 20, 23, 25, 28, 3...",1208


# Fixes & Validations

## ANSWER_LOCATIONS

In [20]:
unknwon_features_df["ANSWER_LOCATIONS"].unique()

array(['[(930, 414), (180, 756), (1680, 756), (930, 1098)]',
       '[(930,414), (180,756), (1680,756), (930,1098)]'], dtype=object)

## SELECT_TO_CONFIRM_RT

check that SELECT_TO_CONFIRM_RT + SELECT_FINAL_ANSWER_RT = CONFIRM_FINAL_ANSWER_RT


In [10]:
(
    unknwon_features_df["SELECT_TO_CONFIRM_RT"]
    + unknwon_features_df["SELECT_FINAL_ANSWER_RT"]
    - unknwon_features_df["CONFIRM_FINAL_ANSWER_RT"]
).value_counts()

-1    12055
 0    11991
Name: count, dtype: int64

In [11]:
len(unknwon_features_df)

24046

In [13]:
unknwon_features_df.iloc[2]["ALL_ANSWERS"]

"[('3', 9422), ('0', 12230), ('2', 13087), ('1', 13569), ('3', 1845), ('1', 3243), ('2', 4342), ('0', 197"